# stats.ipynb

Perform the main statistical analyses

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import analysis
from metadata import METADATA

In [2]:
# Progress bars for apply
tqdm.pandas()

In [3]:
OUTPUT_DIR = Path('results')

## Set up R

In [4]:
%load_ext rpy2.ipython

In [5]:
%%R
library(tidyverse)
library(ggplot2)
library(lme4)
library(broom)
library(multcomp)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

R[write to console]: ✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Loading required package: Matrix

R[write to console]: 
Attaching package: ‘Matrix’


R[write to console]: The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


R[write to console]: Loading required package: mvtnorm

R[write to console]: Loading required package: survival

R[write to console]: Loading required package: TH.data

R[write to console]: Loading required package: MASS

R[write to console]: 
Attaching package: ‘MASS’


R[write to console]: The following object is masked from ‘package:dp

## Build contrast matrices

NOTE: The order of terms is hardcoded. It must match the order in the R model object.

### Main analyses

In [6]:
def rename_terms(model):
    terms = model.loc[model.group == "fixed", "term"]
    terms = terms.str.replace("area", "").str.replace("layer", "").str.replace("stimulus_type", "")
    return terms

In [7]:
terms = [
    "(Intercept)",
    "L4",
    "L5",
    "natural",
    "L4:natural",
    "L5:natural"
]
contrasts = [
    "L2/3:natural - L2/3:artificial",
    "L4:natural - L4:artificial",
    "L5:natural - L5:artificial",
]
LAYER_CONTRAST_MATRIX_MAIN = pd.DataFrame(
    np.zeros([len(contrasts), len(terms)], dtype=int), 
    columns=terms,
    index=contrasts
)
# L23:natural - L23:artificial == 0
#   (intercept + natural) - (intercept) == 0
#   L23:natural == 0
LAYER_CONTRAST_MATRIX_MAIN.loc["L2/3:natural - L2/3:artificial", ["natural"]] = 1
# L4:natural - L4:artificial == 0
#   (intercept + L4 + natural + L4:natural) - (intercept + L4) == 0
#   natural + L4:natural == 0
LAYER_CONTRAST_MATRIX_MAIN.loc["L4:natural - L4:artificial", ["natural", "L4:natural"]] = 1
# L5:natural - L5:artificial == 0
#   (intercept + L5 + natural + L5:natural) - (intercept + L5) == 0
#   natural + L5:natural == 0
LAYER_CONTRAST_MATRIX_MAIN.loc["L5:natural - L5:artificial", ["natural", "L5:natural"]] = 1

LAYER_CONTRAST_MATRIX_MAIN

,(Intercept),L4,L5,natural,L4:natural,L5:natural
L2/3:natural - L2/3:artificial,0,0,0,1,0,0
L4:natural - L4:artificial,0,0,0,1,1,0
L5:natural - L5:artificial,0,0,0,1,0,1


In [8]:
terms = [
    "(Intercept)",
    "LM",
    "AL",
    "PM",
    "AM",
    "natural",
    "LM:natural",
    "AL:natural",
    "PM:natural",
    "AM:natural",
]
contrasts = [
    "V1:natural - V1:artificial",
    "LM:natural - LM:artificial",
    "AL:natural - AL:artificial",
    "PM:natural - PM:artificial",
    "AM:natural - AM:artificial",
]
AREA_CONTRAST_MATRIX_MAIN = pd.DataFrame(
    np.zeros([5, len(terms)], dtype=int), 
    columns=terms,
    index=contrasts,
)
# V1:natural - V1:artificial == 0
#   (intercept + natural + V1 + V1:natural) - (intercept + V1) == 0
#   natural + V1:natural== 0
AREA_CONTRAST_MATRIX_MAIN.loc["V1:natural - V1:artificial", ["natural"]] = 1
# L:natural - L:artificial == 0
#   (intercept + natural + L + L:natural) - (intercept + L) == 0
#   natural + L:natural == 0
AREA_CONTRAST_MATRIX_MAIN.loc["LM:natural - LM:artificial", ["natural", "LM:natural"]] = 1
# AL:natural - AL:artificial == 0
#   (intercept + natural) - (intercept) == 0
#   natural == 0
AREA_CONTRAST_MATRIX_MAIN.loc["AL:natural - AL:artificial", ["natural", "AL:natural"]] = 1
# PM:natural - PM:artificial == 0
#   (intercept + natural + PM + PM:natural) - (intercept + PM) == 0
#   natural + PM:natural == 0
AREA_CONTRAST_MATRIX_MAIN.loc["PM:natural - PM:artificial", ["natural", "PM:natural"]] = 1
# AM:natural - AM:artificial == 0
#   (intercept + natural + AM + AM:natural) - (intercept + AM) == 0
#   natural + AM:natural == 0
AREA_CONTRAST_MATRIX_MAIN.loc["AM:natural - AM:artificial", ["natural", "AM:natural"]] = 1

AREA_CONTRAST_MATRIX_MAIN

,(Intercept),LM,AL,PM,AM,natural,LM:natural,AL:natural,PM:natural,AM:natural
V1:natural - V1:artificial,0,0,0,0,0,1,0,0,0,0
LM:natural - LM:artificial,0,0,0,0,0,1,1,0,0,0
AL:natural - AL:artificial,0,0,0,0,0,1,0,1,0,0
PM:natural - PM:artificial,0,0,0,0,0,1,0,0,1,0
AM:natural - AM:artificial,0,0,0,0,0,1,0,0,0,1


### Arousal covariates

In [9]:
terms = [
    "(Intercept)",
    "L4",
    "L5",
    "natural",
    "locomotion",
    "normalized_pupil_diameter",
    "L4:natural",
    "L5:natural"
]
contrasts = [
    "L2/3:natural - L2/3:artificial",
    "L4:natural - L4:artificial",
    "L5:natural - L5:artificial",
]
LAYER_CONTRAST_MATRIX_AROUSAL = pd.DataFrame(
    np.zeros([len(contrasts), len(terms)], dtype=int), 
    columns=terms,
    index=contrasts
)
# L23:natural - L23:artificial == 0
#   (intercept + natural) - (intercept) == 0
#   L23:natural == 0
LAYER_CONTRAST_MATRIX_AROUSAL.loc["L2/3:natural - L2/3:artificial", ["natural"]] = 1
# L4:natural - L4:artificial == 0
#   (intercept + L4 + natural + L4:natural) - (intercept + L4) == 0
#   natural + L4:natural == 0
LAYER_CONTRAST_MATRIX_AROUSAL.loc["L4:natural - L4:artificial", ["natural", "L4:natural"]] = 1
# L5:natural - L5:artificial == 0
#   (intercept + L5 + natural + L5:natural) - (intercept + L5) == 0
#   natural + L5:natural == 0
LAYER_CONTRAST_MATRIX_AROUSAL.loc["L5:natural - L5:artificial", ["natural", "L5:natural"]] = 1

LAYER_CONTRAST_MATRIX_AROUSAL

,(Intercept),L4,L5,natural,locomotion,normalized_pupil_diameter,L4:natural,L5:natural
L2/3:natural - L2/3:artificial,0,0,0,1,0,0,0,0
L4:natural - L4:artificial,0,0,0,1,0,0,1,0
L5:natural - L5:artificial,0,0,0,1,0,0,0,1


In [10]:
terms = [
    "(Intercept)",
    "LM",
    "AL",
    "PM",
    "AM",
    "natural",
    "locomotion",
    "normalized_pupil_diameter",
    "LM:natural",
    "AL:natural",
    "PM:natural",
    "AM:natural",
]
contrasts = [
    "V1:natural - V1:artificial",
    "LM:natural - LM:artificial",
    "AL:natural - AL:artificial",
    "PM:natural - PM:artificial",
    "AM:natural - AM:artificial",
]
AREA_CONTRAST_MATRIX_AROUSAL = pd.DataFrame(
    np.zeros([5, len(terms)], dtype=int), 
    columns=terms,
    index=contrasts,
)
# V1:natural - V1:artificial == 0
#   (intercept + natural + V1 + V1:natural) - (intercept + V1) == 0
#   natural + V1:natural== 0
AREA_CONTRAST_MATRIX_AROUSAL.loc["V1:natural - V1:artificial", ["natural"]] = 1
# L:natural - L:artificial == 0
#   (intercept + natural + L + L:natural) - (intercept + L) == 0
#   natural + L:natural == 0
AREA_CONTRAST_MATRIX_AROUSAL.loc["LM:natural - LM:artificial", ["natural", "LM:natural"]] = 1
# AL:natural - AL:artificial == 0
#   (intercept + natural) - (intercept) == 0
#   natural == 0
AREA_CONTRAST_MATRIX_AROUSAL.loc["AL:natural - AL:artificial", ["natural", "AL:natural"]] = 1
# PM:natural - PM:artificial == 0
#   (intercept + natural + PM + PM:natural) - (intercept + PM) == 0
#   natural + PM:natural == 0
AREA_CONTRAST_MATRIX_AROUSAL.loc["PM:natural - PM:artificial", ["natural", "PM:natural"]] = 1
# AM:natural - AM:artificial == 0
#   (intercept + natural + AM + AM:natural) - (intercept + AM) == 0
#   natural + AM:natural == 0
AREA_CONTRAST_MATRIX_AROUSAL.loc["AM:natural - AM:artificial", ["natural", "AM:natural"]] = 1

AREA_CONTRAST_MATRIX_AROUSAL

,(Intercept),LM,AL,PM,AM,natural,locomotion,normalized_pupil_diameter,LM:natural,AL:natural,PM:natural,AM:natural
V1:natural - V1:artificial,0,0,0,0,0,1,0,0,0,0,0,0
LM:natural - LM:artificial,0,0,0,0,0,1,0,0,1,0,0,0
AL:natural - AL:artificial,0,0,0,0,0,1,0,0,0,1,0,0
PM:natural - PM:artificial,0,0,0,0,0,1,0,0,0,0,1,0
AM:natural - AM:artificial,0,0,0,0,0,1,0,0,0,0,0,1


## LME functions

In [11]:
%%R
compare_lme_models = function(data, formula, reduced_formula, contrast_matrix, posthoc_alternative) {
    # Estimate models
    model = lmer(formula, data=data, REML=FALSE)
    reduced_model = lmer(reduced_formula, data=data, REML=FALSE)
    # Compare models
    model_comparison = anova(model, reduced_model)
    
    # Run post hoc tests
    posthoc_model = glht(model, as.matrix(contrast_matrix), alternative = posthoc_alternative)
    posthoc_confidence_intervals = confint(posthoc_model)
    
    tidy_model = tidy(model)
    tidy_reduced_model = tidy(reduced_model)
    tidy_comparison = tidy(model_comparison)
    tidy_posthoc = tidy(summary(posthoc_model))
    tidy_posthoc_confidence_intervals = tidy(posthoc_confidence_intervals)

    tidy_comparison$test_type = "likelihood_ratio"
    tidy_posthoc$test_type = "posthoc"
    
    # Return output in tidy form
    list(
        "model" = tidy_model,
        "reduced_model" = tidy_reduced_model,
        "comparison" = tidy_comparison,
        "posthoc" = tidy_posthoc,
        "comparison_description" = capture.output(print(model_comparison)),
        "posthoc_description" = capture.output(print(summary(posthoc_model))),
        "posthoc_confidence_intervals" = capture.output(print(posthoc_confidence_intervals))
    )
}

In [12]:
import rpy2.robjects as ro


def lme_result_to_python(lme_result):
    """Convert the output of the R function `compare_lme_models` to Python objects."""
    with ro.conversion.localconverter(ro.default_converter + ro.pandas2ri.converter):
        output = dict(
            zip(
                ["model", "reduced_model", "comparison", "posthoc"],
                list(map(ro.conversion.rpy2py, lme_result[:4])),
            )
        )
    output.update(
        {
            "comparison_description": "\n".join(lme_result[4]),
            "posthoc_description": "\n".join(lme_result[5]),
            "posthoc_confidence_intervals": "\n".join(lme_result[6]),
        }
    )
    return output

In [13]:
def run_lme(data, formula, reduced_formula, contrast_matrix, posthoc_alternative):
    """
    Keyword Arguments:
        posthoc_alternative: The type of post-hoc test to perform ('two.sided', 'less', 'greater')
    """
    lme_result = %R -i data -i formula -i reduced_formula -i contrast_matrix -i posthoc_alternative compare_lme_models(data, formula, reduced_formula, contrast_matrix, posthoc_alternative)
    # Check that contrast matrix terms are as expected
    lme_result = lme_result_to_python(lme_result)
    expected_terms = list(contrast_matrix.columns)
    actual_terms = list(rename_terms(lme_result['model']))
    assert expected_terms == actual_terms, (
        'Unexpected terms in the posthoc model; check the contrast matrix!\n'
        f'  Expected: {expected_terms}\n'
        f'    Actual: {actual_terms}'
    )
    return lme_result

In [14]:
EXPECTED_NUMBER_OF_OBSERVATIONS = 2200

In [15]:
def run_stats(
    data,
    output_path,
    prefix="",
):
    assert (
        data.shape[0] == EXPECTED_NUMBER_OF_OBSERVATIONS
    ), "Unexpected number of observations, check parameters!"

    posthoc_alternative = "greater"

    layer_result = run_lme(
        data,
        LAYER_FORMULA,
        LAYER_REDUCED_FORMULA,
        LAYER_CONTRAST_MATRIX,
        posthoc_alternative=posthoc_alternative,
    )
    layer_result_df = layer_result["comparison"].merge(
        layer_result["posthoc"], how="outer"
    )
    layer_result_df["interaction"] = "layer"
    layer_result_df["formula"] = LAYER_FORMULA
    layer_result_df["reduced_formula"] = LAYER_REDUCED_FORMULA

    area_result = run_lme(
        data,
        AREA_FORMULA,
        AREA_REDUCED_FORMULA,
        AREA_CONTRAST_MATRIX,
        posthoc_alternative=posthoc_alternative,
    )
    area_result_df = area_result["comparison"].merge(
        area_result["posthoc"], how="outer"
    )
    area_result_df["interaction"] = "area"
    area_result_df["formula"] = AREA_FORMULA
    area_result_df["reduced_formula"] = AREA_REDUCED_FORMULA

    result_df = pd.concat([layer_result_df, area_result_df], axis="rows")
    result_df["posthoc_alternative"] = posthoc_alternative

    with output_path.open(mode="wt") as f:
        f.write(
            prefix
            + "\n".join(
                [
                    "=" * 80,
                    "Layer",
                    "=" * 80,
                    layer_result["comparison_description"],
                    layer_result["posthoc_description"],
                    layer_result["posthoc_confidence_intervals"],
                    "",
                    "=" * 80,
                    "Area",
                    "=" * 80,
                    area_result["comparison_description"],
                    area_result["posthoc_description"],
                    area_result["posthoc_confidence_intervals"],
                    "\n"*2,
                    "~"*80,
                    "\n"*2,
                ]
            )
        )

    return result_df

In [16]:
def run_all_stats(
    group, 
    grouping_parameters=(), 
    output_dir='', 
    filename_prefix="spectral-differentiation", 
    *args, 
    **kwargs,
):
    if len(grouping_parameters) > 1:
        params = dict(zip(grouping_parameters, group.name))
    else:
        params = {grouping_parameters[0]: group.name}
    output_path = (output_dir / ("__".join([filename_prefix, analysis.make_filename(params)]) + '.txt'))
    prefix = "\n".join(
        [
            "Parameters:",
            str(params),
            "\n\n",
        ]
    )
    return run_stats(group, output_path, *args, prefix=prefix, **kwargs)

# Spectral differentiation

### Figures 3 and related figures

- Main results
- Sensitivity analysis
- Arousal variables as covariates
- Event data
- DF/F data with transients removed

In [17]:
response = "log(normalized differentiation)"

In [18]:
LAYER_CONTRAST_MATRIX = LAYER_CONTRAST_MATRIX_MAIN

In [19]:
AREA_CONTRAST_MATRIX = AREA_CONTRAST_MATRIX_MAIN

## Load data

In [20]:
df = pd.read_parquet("results/sensitivity_analysis.parquet")

main = pd.read_parquet("results/main.parquet")

main_events = pd.read_parquet("results/main__events.parquet")

main_no_transients = pd.read_parquet("results/main__no-transients.parquet")

In [21]:
for data in [df, main, main_events, main_no_transients]:
    analysis.prepare_data_for_r(data, response)

## Estimate models

In [22]:
LAYER_FORMULA = f"`{response}` ~ 1 + layer * stimulus_type + (1 | session)"
LAYER_REDUCED_FORMULA = f"`{response}` ~ 1 + layer + stimulus_type + (1 | session)" 
print(LAYER_FORMULA)
print(LAYER_REDUCED_FORMULA)

`log(normalized differentiation)` ~ 1 + layer * stimulus_type + (1 | session)
`log(normalized differentiation)` ~ 1 + layer + stimulus_type + (1 | session)


In [23]:
AREA_FORMULA = f"`{response}` ~ 1 + area * stimulus_type + (1 | session)"
AREA_REDUCED_FORMULA = f"`{response}` ~ 1 + area + stimulus_type + (1 | session)" 
print(AREA_FORMULA)
print(AREA_REDUCED_FORMULA)

`log(normalized differentiation)` ~ 1 + area * stimulus_type + (1 | session)
`log(normalized differentiation)` ~ 1 + area + stimulus_type + (1 | session)


In [24]:
LAYER_CONTRAST_MATRIX = LAYER_CONTRAST_MATRIX_MAIN
AREA_CONTRAST_MATRIX = AREA_CONTRAST_MATRIX_MAIN

In [25]:
GROUPING_PARAMETERS = [
    "state_length",
    "metric",
    "window",
    "window_param",
    "overlap",
    "log_frequency",
]

In [26]:
def sensitivity_analysis(data, output_dir):
    # Run for all combinations
    stats = (
        data.reset_index()
        .groupby(GROUPING_PARAMETERS, dropna=False)
        .progress_apply(
            run_all_stats, 
            grouping_parameters=GROUPING_PARAMETERS,
            output_dir=output_dir, 
            filename_prefix='spectral-differentiation',
        )
    )
    stats.index = stats.index.droplevel(None)

    index = stats.index.names
    stats = stats.reset_index()
    # Remap values for display
    stats = stats.assign(
        window=stats['window'].replace({None: 'rectangular'}),
        overlap=stats['overlap'].fillna(0),
    )
    # Rename posthoc differences
    renames = {
        'layer': {
            "L2/3:natural - L2/3:artificial": "L2/3",
            "L4:natural - L4:artificial": "L4",
            "L5:natural - L5:artificial": "L5",
        },
        'area': {
            "V1:natural - V1:artificial": "V1",
            "LM:natural - LM:artificial": "LM",
            "AL:natural - AL:artificial": "AL",
            "PM:natural - PM:artificial": "PM",
            "AM:natural - AM:artificial": "AM",
        }
    }
    for interaction, rename in renames.items():
        stats[interaction] = stats["lhs"].map(rename)
        stats[interaction] = pd.Categorical(
            stats[interaction], categories=rename.values(), ordered=True
        )
    # Restore index
    stats = stats.set_index(index)
    
    return stats

In [27]:
output_dir = OUTPUT_DIR/'stats'
output_dir.mkdir(exist_ok=True, parents=True)

# Unscrambled vs. scrambled
subset = (
    df
    .loc[df["stimulus_is_scrambled_pair"]]
    .reset_index()
)
print(subset.shape)

stats = sensitivity_analysis(subset, output_dir)
stats.to_parquet(OUTPUT_DIR/'stats.parquet')

(237600, 34)


  0%|          | 0/108 [00:00<?, ?it/s]

/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2085: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  values, tz_parsed = conversion.datetime_to_datetime64(data)
/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2201: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  elif not timezones.tz_compare(tz, inferred_tz):
/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:2282: PytzUsageWarning: The zone attribute is specific to pytz's i

### Figure 3

In [28]:
cat results/stats/spectral-differentiation__state_length-1.0__metric-euclidean__window-nan__window_param--1.0__overlap-0.0__log_frequency-False.txt

Parameters:
{'state_length': 1.0, 'metric': 'euclidean', 'window': nan, 'window_param': -1.0, 'overlap': 0.0, 'log_frequency': False}


Layer
Data: data
Models:
reduced_model: `log(normalized differentiation)` ~ 1 + layer + stimulus_type + 
reduced_model:     (1 | session)
model: `log(normalized differentiation)` ~ 1 + layer * stimulus_type + 
model:     (1 | session)
              Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)   
reduced_model  6 2340.0 2374.1 -1164.0   2328.0                            
model          8 2330.6 2376.2 -1157.3   2314.6 13.379      2   0.001244 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = `log(normalized differentiation)` ~ 1 + layer * 
    stimulus_type + (1 | session), data = data, REML = FALSE)

Linear Hypotheses:
                                     Estimate Std. Error z value   Pr(>z)    
L2/3:natural - L2/3:artificial <= 0  0.113259   0.02

### Figure 3-1

In [29]:
subset = main.loc[
    main['stimulus_is_block']
]

In [30]:
output_path = OUTPUT_DIR / "stats__fig_3-1.txt"

In [31]:
EXPECTED_NUMBER_OF_OBSERVATIONS = 5280

In [32]:
result = run_stats(subset, output_path)

In [33]:
cat {output_path}

Layer
Data: data
Models:
reduced_model: `log(normalized differentiation)` ~ 1 + layer + stimulus_type + 
reduced_model:     (1 | session)
model: `log(normalized differentiation)` ~ 1 + layer * stimulus_type + 
model:     (1 | session)
              Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
reduced_model  6 6687.4 6726.8 -3337.7   6675.4                             
model          8 6675.0 6727.6 -3329.5   6659.0 16.343      2  0.0002827 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = `log(normalized differentiation)` ~ 1 + layer * 
    stimulus_type + (1 | session), data = data, REML = FALSE)

Linear Hypotheses:
                                    Estimate Std. Error z value   Pr(>z)    
L2/3:natural - L2/3:artificial <= 0  0.11085    0.02229   4.974 9.82e-07 ***
L4:natural - L4:artificial <= 0     -0.01002    0.02229  -0.450 0.965212    
L5:natural - L5:artificial <= 0 

### Figure 3-8

In [34]:
subset = main_events.loc[
    main_events["stimulus_is_scrambled_pair"]
]

In [35]:
output_path = OUTPUT_DIR / "stats__fig_3-8.txt"

In [36]:
EXPECTED_NUMBER_OF_OBSERVATIONS = 2200

In [37]:
result = run_stats(subset, output_path)

In [38]:
cat {output_path}

Layer
Data: data
Models:
reduced_model: `log(normalized differentiation)` ~ 1 + layer + stimulus_type + 
reduced_model:     (1 | session)
model: `log(normalized differentiation)` ~ 1 + layer * stimulus_type + 
model:     (1 | session)
              Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)   
reduced_model  6 2112.9 2147.1 -1050.4   2100.9                            
model          8 2105.4 2151.0 -1044.7   2089.4 11.481      2   0.003213 **
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = `log(normalized differentiation)` ~ 1 + layer * 
    stimulus_type + (1 | session), data = data, REML = FALSE)

Linear Hypotheses:
                                    Estimate Std. Error z value   Pr(>z)    
L2/3:natural - L2/3:artificial <= 0  0.10702    0.02791   3.835 0.000188 ***
L4:natural - L4:artificial <= 0     -0.00431    0.02791  -0.154 0.915610    
L5:natural - L5:artificial <= 0    

### Figure 3-9

In [39]:
subset = main_no_transients.loc[
    main_events["stimulus_is_scrambled_pair"]
]

In [40]:
output_path = OUTPUT_DIR / "stats__fig_3-9.txt"

In [41]:
EXPECTED_NUMBER_OF_OBSERVATIONS = 2200

In [42]:
result = run_stats(subset, output_path)

In [43]:
cat {output_path}

Layer
Data: data
Models:
reduced_model: `log(normalized differentiation)` ~ 1 + layer + stimulus_type + 
reduced_model:     (1 | session)
model: `log(normalized differentiation)` ~ 1 + layer * stimulus_type + 
model:     (1 | session)
              Df    AIC    BIC  logLik deviance Chisq Chi Df Pr(>Chisq)  
reduced_model  6 1332.5 1366.7 -660.26   1320.5                          
model          8 1330.5 1376.1 -657.25   1314.5 6.024      2    0.04919 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = `log(normalized differentiation)` ~ 1 + layer * 
    stimulus_type + (1 | session), data = data, REML = FALSE)

Linear Hypotheses:
                                    Estimate Std. Error z value  Pr(>z)   
L2/3:natural - L2/3:artificial <= 0  0.06634    0.02350   2.823 0.00713 **
L4:natural - L4:artificial <= 0      0.01998    0.02350   0.850 0.48337   
L5:natural - L5:artificial <= 0     -0.01640   

## Figures 3-5, 3-6, & 3-7

Sensitivity analysis with arousal covariates

In [44]:
LAYER_CONTRAST_MATRIX = LAYER_CONTRAST_MATRIX_AROUSAL
AREA_CONTRAST_MATRIX = AREA_CONTRAST_MATRIX_AROUSAL

### Estimate models

In [45]:
LAYER_FORMULA = f"`{response}` ~ 1 + layer * stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)"
LAYER_REDUCED_FORMULA = f"`{response}` ~ 1 + layer + stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)" 
print(LAYER_FORMULA)
print(LAYER_REDUCED_FORMULA)

`log(normalized differentiation)` ~ 1 + layer * stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)
`log(normalized differentiation)` ~ 1 + layer + stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)


In [46]:
AREA_FORMULA = f"`{response}` ~ 1 + area * stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)"
AREA_REDUCED_FORMULA = f"`{response}` ~ 1 + area + stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)" 
print(AREA_FORMULA)
print(AREA_REDUCED_FORMULA)

`log(normalized differentiation)` ~ 1 + area * stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)
`log(normalized differentiation)` ~ 1 + area + stimulus_type + locomotion + normalized_pupil_diameter + (1 | session)


In [47]:
output_dir = OUTPUT_DIR/'stats__arousal'
output_dir.mkdir(exist_ok=True, parents=True)

# Unscrambled vs. scrambled
subset = (
    df
    .loc[df["stimulus_is_scrambled_pair"]]
    .reset_index()
)
print(subset.shape)

stats = sensitivity_analysis(subset, output_dir)
stats.to_parquet(OUTPUT_DIR/'stats__arousal.parquet')

(237600, 34)


  0%|          | 0/108 [00:00<?, ?it/s]

# Mean centroid distance

In [48]:
response = "log(mean_centroid_distance)"

In [49]:
df = pd.read_parquet("results/mean_centroid_distance.parquet")

df_events = pd.read_parquet("results/mean_centroid_distance__events.parquet")

In [50]:
for data in [df, df_events]:
    analysis.prepare_data_for_r(data, response)

In [51]:
LAYER_FORMULA = f"`{response}` ~ 1 + layer * stimulus_type + (1 | session)"
LAYER_REDUCED_FORMULA = f"`{response}` ~ 1 + layer + stimulus_type + (1 | session)" 
print(LAYER_FORMULA)
print(LAYER_REDUCED_FORMULA)

`log(mean_centroid_distance)` ~ 1 + layer * stimulus_type + (1 | session)
`log(mean_centroid_distance)` ~ 1 + layer + stimulus_type + (1 | session)


In [52]:
AREA_FORMULA = f"`{response}` ~ 1 + area * stimulus_type + (1 | session)"
AREA_REDUCED_FORMULA = f"`{response}` ~ 1 + area + stimulus_type + (1 | session)" 
print(AREA_FORMULA)
print(AREA_REDUCED_FORMULA)

`log(mean_centroid_distance)` ~ 1 + area * stimulus_type + (1 | session)
`log(mean_centroid_distance)` ~ 1 + area + stimulus_type + (1 | session)


In [53]:
LAYER_CONTRAST_MATRIX = LAYER_CONTRAST_MATRIX_MAIN
AREA_CONTRAST_MATRIX = AREA_CONTRAST_MATRIX_MAIN

In [54]:
EXPECTED_NUMBER_OF_OBSERVATIONS = 2200

### Figure 5

In [55]:
subset = df.loc[
    df["stimulus_is_scrambled_pair"]
]

In [56]:
output_path = OUTPUT_DIR / "stats__fig_5.txt"

In [57]:
result = run_stats(subset, output_path)

In [58]:
cat {output_path}

Layer
Data: data
Models:
reduced_model: `log(mean_centroid_distance)` ~ 1 + layer + stimulus_type + (1 | 
reduced_model:     session)
model: `log(mean_centroid_distance)` ~ 1 + layer * stimulus_type + (1 | 
model:     session)
              Df     AIC     BIC logLik deviance  Chisq Chi Df Pr(>Chisq)    
reduced_model  6 -4656.7 -4622.5 2334.4  -4668.7                             
model          8 -4670.9 -4625.3 2343.4  -4686.9 18.135      2  0.0001154 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = `log(mean_centroid_distance)` ~ 1 + layer * stimulus_type + 
    (1 | session), data = data, REML = FALSE)

Linear Hypotheses:
                                     Estimate Std. Error z value   Pr(>z)    
L2/3:natural - L2/3:artificial <= 0  0.030702   0.005962   5.149 3.92e-07 ***
L4:natural - L4:artificial <= 0      0.010430   0.005962   1.749    0.116    
L5:natural - L5:artificial <= 0     -0

### Figure 5-1

In [59]:
subset = df_events.loc[
    df_events["stimulus_is_scrambled_pair"]
]

In [60]:
output_path = OUTPUT_DIR / "stats__fig_5-1.txt"

In [61]:
EXPECTED_NUMBER_OF_OBSERVATIONS = 2200

In [62]:
result = run_stats(subset, output_path)

In [63]:
cat {output_path}

Layer
Data: data
Models:
reduced_model: `log(mean_centroid_distance)` ~ 1 + layer + stimulus_type + (1 | 
reduced_model:     session)
model: `log(mean_centroid_distance)` ~ 1 + layer * stimulus_type + (1 | 
model:     session)
              Df     AIC     BIC logLik deviance  Chisq Chi Df Pr(>Chisq)    
reduced_model  6 -5449.1 -5414.9 2730.5  -5461.1                             
model          8 -5460.1 -5414.5 2738.1  -5476.1 15.029      2  0.0005452 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

	 Simultaneous Tests for General Linear Hypotheses

Fit: lmer(formula = `log(mean_centroid_distance)` ~ 1 + layer * stimulus_type + 
    (1 | session), data = data, REML = FALSE)

Linear Hypotheses:
                                      Estimate Std. Error z value   Pr(>z)    
L2/3:natural - L2/3:artificial <= 0  0.0262792  0.0049239   5.337 1.42e-07 ***
L4:natural - L4:artificial <= 0      0.0083611  0.0049239   1.698    0.128    
L5:natural - L5:artificial <= 0    

# Arousal mediation analyses

In [64]:
%%R 

install.packages('mediation', repos='https://cloud.r-project.org/')
library(mediation)

R[write to console]: also installing the dependencies ‘png’, ‘jpeg’, ‘checkmate’, ‘htmlwidgets’, ‘Formula’, ‘latticeExtra’, ‘gridExtra’, ‘data.table’, ‘htmlTable’, ‘viridis’, ‘lpSolve’, ‘Hmisc’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/png_0.1-7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 24990 bytes (24 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write t

x86_64-conda-linux-gnu-cc -I"/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -I/home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -Wl,-rpath-link,/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib  `libpng-config --cflags` -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base_1621283253293/work=/usr/local/src/conda/r-base-3.6.3 -fdebug-prefix-map=/home/wmayner/miniconda3/envs/openscope-differentiation-test3=/usr/local/src/conda-prefix  -c read.c -o read.o
x86_64-conda-linux-gnu-cc -I"/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_

installing to /home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/library/00LOCK-png/00new/png/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (png)
* installing *source* package ‘jpeg’ ...
** package ‘jpeg’ successfully unpacked and MD5 sums checked
** using staged installation
** libs


x86_64-conda-linux-gnu-cc -I"/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -I/home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -Wl,-rpath-link,/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib  -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base_1621283253293/work=/usr/local/src/conda/r-base-3.6.3 -fdebug-prefix-map=/home/wmayner/miniconda3/envs/openscope-differentiation-test3=/usr/local/src/conda-prefix  -c read.c -o read.o
x86_64-conda-linux-gnu-cc -I"/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /ho

installing to /home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/library/00LOCK-jpeg/00new/jpeg/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (jpeg)
* installing *source* package ‘checkmate’ ...
** package ‘checkmate’ successfully unpacked and MD5 sums checked
** using staged installation
** libs


...

In file included from lp_rlp.c:97:
lp_rlp.h: In function 'lp_yy_fatal_error':
lp_rlp.h:1804:19: warning: passing argument 1 of 'lex_fatal_error' discards 'const' qualifier from pointer target type [-Wdiscarded-qualifiers]
 1804 |  lex_fatal_error (msg); /* exit( YY_EXIT_FAILURE ); */
      |                   ^~~
In file included from lp_rlp.c:32:
yacc_read.h:6:28: note: expected 'char *' but argument is of type 'const char *'
    6 | void lex_fatal_error(char *msg);
      |                      ~~~~~~^~~


x86_64-conda-linux-gnu-cc -I"/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/include" -DNDEBUG -I . -DINTEGERTIME -DPARSER_LP -DBUILDING_FOR_R -DYY_NEVER_INTERACTIVE -DUSRDLL -DCLOCKTIME -DRoleIsExternalInvEngine -DINVERSE_ACTIVE=INVERSE_LUSOL -DINLINE=static -DParanoia  -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -I/home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -Wl,-rpath-link,/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib  -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base_1621283253293/work=/usr/local/src/conda/r-base-3.6.3 -fdebug-prefix-map=/home/wmayner/miniconda3/envs/openscope-differentiation-test3=/usr/local/src/conda-prefix  -c lp_s

installing to /home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/library/00LOCK-lpSolve/00new/lpSolve/libs
** R
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (lpSolve)
* installing *source* package ‘latticeExtra’ ...
** package ‘latticeExtra’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location


x86_64-conda-linux-gnu-cc -I"/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -I/home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -Wl,-rpath-link,/home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib  -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/wmayner/miniconda3/envs/openscope-differentiation-test3/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base_1621283253293/work=/usr/local/src/conda/r-base-3.6.3 -fdebug-prefix-map=/home/wmayner/miniconda3/envs/openscope-differentiation-test3=/usr/local/src/conda-prefix  -c Hmisc.c -o Hmisc.o
x86_64-conda-linux-gnu-gfortran -fno-optimize-sibling-calls  -fpic  -fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-p

installing to /home/wmayner/miniconda3/envs/openscope-differentiation-test3/lib/R/library/00LOCK-Hmisc/00new/Hmisc/libs
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (Hmisc)
* installing *source* package ‘mediation’ ...
** package ‘mediation’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** data
*** moving datasets to lazyload DB
** demo
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be

In [65]:
%R set.seed(803317503)

In [66]:
data = main.loc[
    (main['stimulus_is_scrambled_pair']) &
    (main['layer'] == 'L2/3') &
    (main['area'].isin(['AL', 'AM']))
]
data.shape

(300, 33)

In [67]:
response = '`log(normalized differentiation)`'

mediator = 'mean_magnitude'

treatment = 'stimulus_type'

covariates = 'normalized_pupil_diameter + locomotion'

In [68]:
mediator_formula = f"{mediator} ~ 1 + {treatment} + {covariates} + (1 | session)"
outcome_formula = f"{response} ~ 1 + {mediator} + {treatment} + {covariates} + (1 | session)"
print(mediator_formula, end='\n\n')
print(outcome_formula)

mean_magnitude ~ 1 + stimulus_type + normalized_pupil_diameter + locomotion + (1 | session)

`log(normalized differentiation)` ~ 1 + mean_magnitude + stimulus_type + normalized_pupil_diameter + locomotion + (1 | session)


In [69]:
%%R -i data -i mediator_formula -i outcome_formula

mediator_model = lmer(mediator_formula, data=data, REML=FALSE)
outcome_model = lmer(outcome_formula, data=data, REML=FALSE)

In [70]:
%%R -i treatment -i mediator
med = mediate(mediator_model, outcome_model, treat=treatment, mediator=mediator, robustSE=TRUE)
print(summary(med))


Causal Mediation Analysis 

Quasi-Bayesian Confidence Intervals

Mediator Groups: session 

Outcome Groups: session 

Output Based on Overall Averages Across Groups 

               Estimate 95% CI Lower 95% CI Upper p-value    
ACME             0.1351       0.0784         0.20  <2e-16 ***
ADE              0.0951       0.0400         0.15  <2e-16 ***
Total Effect     0.2302       0.1516         0.31  <2e-16 ***
Prop. Mediated   0.5898       0.3963         0.80  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Sample Size Used: 300 


Simulations: 1000 

